<a href="https://colab.research.google.com/github/Nagendra0228/INS-Task1/blob/main/Task_1_Cipher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Playfair Cipher (Encryption & Decryption)

In [1]:
import numpy as np

# Function to generate the 5x5 Playfair matrix
def generate_playfair_matrix(key):
    key = "".join(dict.fromkeys(key.replace("J", "I")))  # Remove duplicates, replace J with I
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    matrix = key + "".join([c for c in alphabet if c not in key])
    return np.array(list(matrix)).reshape(5, 5)

# Function to find row and column of a letter in the matrix
def find_position(matrix, letter):
    row, col = np.where(matrix == letter)
    return row[0], col[0]

# Function to encrypt digraphs
def encrypt_playfair(text, matrix):
    text = text.replace("J", "I").upper()
    text = "".join([c for c in text if c.isalpha()])

    # Convert text into digraphs, inserting 'X' if necessary
    pairs = []
    i = 0
    while i < len(text):
        a = text[i]
        b = text[i+1] if i+1 < len(text) and text[i] != text[i+1] else 'X'
        pairs.append(a + b)
        i += 2 if i+1 < len(text) and text[i] != text[i+1] else 1

    cipher_text = ""
    for pair in pairs:
        r1, c1 = find_position(matrix, pair[0])
        r2, c2 = find_position(matrix, pair[1])

        if r1 == r2:  # Same row
            cipher_text += matrix[r1, (c1+1) % 5] + matrix[r2, (c2+1) % 5]
        elif c1 == c2:  # Same column
            cipher_text += matrix[(r1+1) % 5, c1] + matrix[(r2+1) % 5, c2]
        else:  # Rectangle swap
            cipher_text += matrix[r1, c2] + matrix[r2, c1]

    return cipher_text

# Function to decrypt
def decrypt_playfair(text, matrix):
    plain_text = ""
    for i in range(0, len(text), 2):
        r1, c1 = find_position(matrix, text[i])
        r2, c2 = find_position(matrix, text[i+1])

        if r1 == r2:  # Same row
            plain_text += matrix[r1, (c1-1) % 5] + matrix[r2, (c2-1) % 5]
        elif c1 == c2:  # Same column
            plain_text += matrix[(r1-1) % 5, c1] + matrix[(r2-1) % 5, c2]
        else:  # Rectangle swap
            plain_text += matrix[r1, c2] + matrix[r2, c1]

    return plain_text

# Example Usage
key = "PLAYFAIR"
matrix = generate_playfair_matrix(key)
plaintext = "HELLO"
ciphertext = encrypt_playfair(plaintext, matrix)
decrypted = decrypt_playfair(ciphertext, matrix)

print("Playfair Cipher")
print("Encrypted:", ciphertext)
print("Decrypted:", decrypted)


Playfair Cipher
Encrypted: KGYVRV
Decrypted: HELXLO


2. Hill Cipher (Encryption & Decryption)

In [4]:
import numpy as np

# Function to convert text to numerical values
def text_to_numbers(text):
    return [ord(c) - ord('A') for c in text]

# Function to convert numerical values back to text
def numbers_to_text(numbers):
    return ''.join(chr(n + ord('A')) for n in numbers)

# Encrypt using Hill Cipher
def encrypt_hill(plaintext, key_matrix):
    n = len(key_matrix)
    plaintext = plaintext.upper().replace(" ", "")
    plaintext_numbers = text_to_numbers(plaintext)

    # Pad plaintext if necessary
    while len(plaintext_numbers) % n != 0:
        plaintext_numbers.append(0)

    # Convert plaintext into matrix form
    plaintext_matrix = np.array(plaintext_numbers).reshape(-1, n)

    # Encrypt using matrix multiplication
    cipher_matrix = (np.dot(plaintext_matrix, key_matrix) % 26).flatten()
    return numbers_to_text(cipher_matrix)

# Decrypt using Hill Cipher
def decrypt_hill(ciphertext, key_matrix):
    n = len(key_matrix)
    ciphertext_numbers = text_to_numbers(ciphertext)

    # Convert ciphertext into matrix form
    cipher_matrix = np.array(ciphertext_numbers).reshape(-1, n)

    # Compute modular inverse of the key matrix
    det = int(np.round(np.linalg.det(key_matrix))) % 26
    det_inv = pow(det, -1, 26)  # Modular inverse under mod 26
    adjugate = np.round(det * np.linalg.inv(key_matrix)).astype(int) % 26
    inverse_matrix = (det_inv * adjugate) % 26

    # Decrypt using matrix multiplication
    plain_matrix = (np.dot(cipher_matrix, inverse_matrix) % 26).flatten()
    return numbers_to_text(plain_matrix)

# Example Usage
key_matrix = np.array([[3, 3], [2, 5]])  # Invertible mod 26
plaintext = "HI"
ciphertext = encrypt_hill(plaintext, key_matrix)
decrypted = decrypt_hill(ciphertext, key_matrix)

print("\nHill Cipher")
print("Encrypted:", ciphertext)
print("Decrypted:", decrypted)



Hill Cipher
Encrypted: LJ
Decrypted: HI


3. Vigenère Cipher (Encryption & Decryption)

In [5]:
# Function to generate key of the same length as text
def generate_key(text, key):
    key = key.upper()
    return (key * (len(text) // len(key))) + key[:len(text) % len(key)]

# Encrypt using Vigenère Cipher
def encrypt_vigenere(text, key):
    text = text.upper()
    key = generate_key(text, key)
    cipher_text = ""

    for i in range(len(text)):
        shift = ord(key[i]) - ord('A')
        cipher_text += chr(((ord(text[i]) - ord('A') + shift) % 26) + ord('A'))

    return cipher_text

# Decrypt using Vigenère Cipher
def decrypt_vigenere(text, key):
    text = text.upper()
    key = generate_key(text, key)
    plain_text = ""

    for i in range(len(text)):
        shift = ord(key[i]) - ord('A')
        plain_text += chr(((ord(text[i]) - ord('A') - shift) % 26) + ord('A'))

    return plain_text

# Example Usage
plaintext = "HELLO"
key = "KEY"
ciphertext = encrypt_vigenere(plaintext, key)
decrypted = decrypt_vigenere(ciphertext, key)

print("\nVigenère Cipher")
print("Encrypted:", ciphertext)
print("Decrypted:", decrypted)



Vigenère Cipher
Encrypted: RIJVS
Decrypted: HELLO
